In [1]:
import numpy as np
import cv2 as cv
import time
from pathlib import Path
import depthai as dai


"""
    Captures grayscale images from a mono camera connected to the specified board socket.

    :param board_socket: Socket on the device where the camera is connected ("left" or "right").
    :param delay_ms: Delay in milliseconds between capturing consecutive images.
    :param num_images: Number of images to capture.
    """
def capture_images_from_mono_camera(board_socket="right", delay_ms=2000, num_images=10):

    if board_socket not in ["left", "right"]:
        print("Invalid board socket specified. Use 'left' or 'right'.")
        return

    # Define resolution for the mono camera
    sensor_resolution = dai.MonoCameraProperties.SensorResolution.THE_480_P

    # Create a pipeline
    pipeline = dai.Pipeline()

    # Define a mono camera node
    cam = pipeline.createMonoCamera()
    cam.setResolution(sensor_resolution)

    # Set the board socket for the camera
    if board_socket == "right":
        cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    else:
        cam.setBoardSocket(dai.CameraBoardSocket.LEFT)

    # Create XLink output for the camera
    xout = pipeline.createXLinkOut()
    xout.setStreamName(board_socket)
    cam.out.link(xout.input)

    # Connect to the device and start the pipeline
    with dai.Device(pipeline) as device:

        # Get the output queue for receiving frames
        q = device.getOutputQueue(name=board_socket, maxSize=4, blocking=False)

        # Create directory for saving images
        image_dir = Path(f"{board_socket}")
        image_dir.mkdir(parents=True, exist_ok=True)

        # Capture specified number of images
        for _ in range(num_images):
            # Get frame from the queue
            in_frame = q.get()
            frame = in_frame.getCvFrame()

            # Display and save the image
            cv.imshow(board_socket, frame)
            cv.imwrite(str(image_dir / f"{int(time.time() * 10000)}.jpg"), frame)
            cv.waitKey(delay_ms)

        cv.destroyAllWindows()
        device.close()




"""
    Captures color images from a color camera.

    :param delay_ms: Delay in milliseconds between capturing consecutive images.
    :param num_images: Number of images to capture.
    """
def capture_images_from_color_camera(delay_ms=2000, num_images=10):

    # Define resolution for the color camera
    sensor_resolution = dai.ColorCameraProperties.SensorResolution.THE_1080_P

    # Create a pipeline
    pipeline = dai.Pipeline()

    # Define a color camera node
    cam = pipeline.createColorCamera()
    cam.setResolution(sensor_resolution)

    # Create XLink output for the camera
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect to the device and start the pipeline
    with dai.Device(pipeline) as device:

        # Get the output queue for receiving frames
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Create directory for saving images
        image_dir = Path("rgb")
        image_dir.mkdir(parents=True, exist_ok=True)

        # Capture specified number of images
        for _ in range(num_images):
            # Get frame from the queue
            in_frame = q.get()
            frame = in_frame.getCvFrame()

            # Display and save the image
            imS = cv.resize(frame, (960, 540))  # Resize image
            cv.imshow("rgb", imS)
            cv.imwrite(str(image_dir / f"{int(time.time() * 10000)}.jpg"), frame)
            cv.waitKey(delay_ms)

        cv.destroyAllWindows()
        device.close()

In [17]:
# capturing images using right monochrome camera
capture_images_from_mono_camera('right')

# capturing images using left monochrome camera
capture_images_from_mono_camera('left')

# capturing images using rgb camera
capture_images_from_color_camera(delay_ms=2000, num_images=10)

/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_34725/2234083928.py:33: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_34725/2234083928.py:35: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.LEFT)


In [28]:
def calibrate_camera(objpoints, imgpoints, image_dir, camera_type):
    # Termination criteria for corner detection
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points (chessboard corners)
    square_size = 0.03  # 3cm
    objp = np.zeros((5 * 8, 3), np.float32)
    objp[:, :2] = np.mgrid[0:8, 0:5].T.reshape(-1, 2) * square_size

    # Arrays to store object points and image points
    objpoints[camera_type] = []  # 3D points in real-world space
    imgpoints[camera_type] = []  # 2D points in image plane

    images = list(Path(image_dir).glob("*.jpg"))

    for fname in images:
        img = cv.imread(str(fname))
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        ret, corners = cv.findChessboardCorners(gray, (8, 5), None)

        if ret:
            objpoints[camera_type].append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints[camera_type].append(corners2)

            # Save corner-detected image in 'corners_<camera_type>' folder
            corners_dir = Path(f"corners_{camera_type}")
            corners_dir.mkdir(parents=True, exist_ok=True)  # Ensure folder exists
            corner_image_path = corners_dir / f"{fname.stem}_corners.jpg"  # Path to save corner-detected image
            cv.drawChessboardCorners(img, (8, 5), corners2, ret)
            cv.imwrite(str(corner_image_path), img)

    return objpoints, imgpoints

In [29]:
def save_matrix(matrix, filename):
    np.savetxt(filename, matrix)

In [32]:
def main():
    objpoints = {}
    imgpoints = {}

    # Specify the directories containing left and right monochrome images and RGB images
    image_dirs = {
        'left': 'left',
        'right': 'right',
        'rgb': 'rgb'
    }

    # Calibrate each type of camera
    for camera_type, image_dir in image_dirs.items():
        objpoints, imgpoints = calibrate_camera(objpoints, imgpoints, image_dir, camera_type)

        if objpoints[camera_type]:
            # Calibrate camera
            ret, mtx, dist, _, _ = cv.calibrateCamera(
                objpoints[camera_type], imgpoints[camera_type], (640, 480), None, None
            )

            # Save calibration matrices
            mtx_file = f"{camera_type}_camera_matrix.txt"
            dist_file = f"{camera_type}_distortion_coefficients.txt"
            save_matrix(mtx, mtx_file)
            save_matrix(dist, dist_file)

            # Get the path to the corners folder for the specified camera type
            corners_dir = Path.cwd() / f"corners_{camera_type}"

            # Get the list of all image files inside the corners folder
            image_files = list(corners_dir.glob("*.jpg"))

            # Solve for intrinsic and extrinsic parameters
            if imgpoints[camera_type]:
                img = cv.imread(str(image_files[0]))
                if img is None:
                    print("Error: Unable to read image from", img_path)
                    return
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                ret, rvecs, tvecs, _ = cv.solvePnPRansac(objpoints[camera_type][0], imgpoints[camera_type][0], mtx, dist)

                # Compute rotation matrix from rotation vectors
                rmat, _ = cv.Rodrigues(rvecs)
                rmat_file = f"{camera_type}_rotation_matrix.txt"
                save_matrix(rmat, rmat_file)

                # Save translation vector
                tvec_file = f"{camera_type}_translation_vector.txt"
                save_matrix(tvecs, tvec_file)

        else:
            print(f"No chessboard corners found for {camera_type} camera images.")

    print("\nCalibration completed successfully!")


if __name__ == "__main__":
    main()


Calibration completed successfully!


For the left camera matrix:

Focal length (fx, fy) = (601.31, 598.40)

For the right camera matrix:

Focal length (fx, fy) = (498.88, 498.45)

Given the translation vectors:

Left translation vector (Tx1) = -9.791114442501212822e-02

Right translation vector (Tx2) = -1.511770870082925766e-01

The baseline can be calculated as:

Baseline = |Tx1 - Tx2|

Substituting the values:

Baseline = |(-9.791114442501212822e-02) - (-1.511770870082925766e-01)|

Baseline = |0.09791114442501212822 - 0.1511770870082925766|

Baseline ≈ 0.05326594258328035438

Baseline ≈ 53.27 millimeters

So, the estimated baseline between the left and right cameras is approximately 0.053 meters.